In [ ]:
!nvidia-smi

In [ ]:
import dask_cudf
import cudf, cuml, cugraph
import json
from pandas.io.json import json_normalize 
import pandas as pd 
import time

In [ ]:
#Fase load PANDAS, RAPIDS and Dask_RAPIDS

pd_Pandas = pd.read_csv('../df_data.csv', names='uno')

#read with cudf
start = time.time()
gdf = cudf.read_csv('../df_data.csv', names='uno')
end = time.time()

caculatedCUDF = end - start
print("Cálculo de lectura con RAPIDS = {}".format(caculatedCUDF))

#read with dask_cudf
daskstart = time.time()
dask_gdf = dask_cudf.read_csv('../df_data.csv', names='uno')
daskend = time.time()

caculatedDASKCUDF = daskend - daskstart
print("Cálculo de lectura con DASK_CUDF = {}".format(caculatedDASKCUDF))

In [ ]:
cudf_Pandas = pd_Pandas
print('Descripción de salida del dataset:')
gdf

In [ ]:
#Fase Transformation
#Dataset more understandable

import json
#crear formato json para despues crear dataframe
start = time.time()
lista = list(cudf_Pandas['u'])
a = []

for i in range(0,len(lista)):
    d = lista[i].split('.gz:')
    nuevo = d[1]
    a.append(json.loads((nuevo)))
    
#Create DataFrame with Pandas
pdPandas = pd.DataFrame(a)

# Convierto todo a str para que la entrega funcione
#Pandas a cudf, si no lo convierto me arroja errores y no puedes
#convertir en un marco de datos
nuevo_pdPandas = pdPandas.applymap(str)

#Crear DataFrane con cudf
gdf = cudf.DataFrame.from_pandas(nuevo_pdPandas)

#convertir para Pandas
gdfPD = gdf.to_pandas()

# solo conservo el día de la columna 'username'
#mayor a 0
snNull = gdfPD[gdfPD.username.map(len) > 0]

# elimino username dicc y opencampus
snDicc = snNull[snNull.username != 'dicc']
snCampus = snDicc[snDicc.username != 'opencampus']

#convertio para cuDF
gdf_new = cudf.DataFrame.from_pandas(snCampus)

#eliminar filas con los datos perdidos
gdf_new.dropna(axis=0, how='any')

#Eliminar columnas que no tiene mucha relevancia
gdf_new.drop_column('accept_language') #columna si relevancia
gdf_new.drop_column('name') #muchos datos faltantes
gdf_new.drop_column('agent') #columna si relevancia
gdf_new.drop_column('page') #muchos datos faltantes
gdf_new.drop_column('session') #muchos datos faltantes
gdf_new.drop_column('event') #datos dañados
gdf_new.drop_column('event_type') #columna si relevancia

#Valores server = 1, browser = 2
gdf_new['event_source'].replace(['server','browser'],
                                [0,1],inplace=True)

#Valor opencampus.utpl.edu.ec = 0
#valor soer.utpl.edu.ec = 1
gdf_new['host'].replace(['opencampus.utpl.edu.ec','soer.utpl.edu.ec'],
                        [0,1],inplace=True)

#ip a formato int
gdf_new['ip'] = gdf_new['ip']
gdf_new['ip'].str.ip2int()

#Dividir columna context estaba en json
context = gdf_new['context']
context.str.normalize_spaces()
newcontext = context.str.split(pat = ",")
newcontext.drop_column(3)
newcontext.drop_column(4)
newcontext.drop_column(5)
newcontext.drop_column(6)

#Cambiar nombre a las columnas
newcontext.columns = ['a','b','c']

#Unir
newData = gdf_new.join(newcontext, how='left')
newData.drop_column('context')

#eliminar filas con datos perdidos
newData.dropna(axis=0, how='any')

#soltar duplicados
newData.sort_values('username') 
newData.drop_duplicates(subset='username', keep=False, inplace=False, ignore_index=False)

#resetear index
newData = newData.reset_index()
newData.drop_column('index')

#eliminar filas con datos perdidos
newData.dropna(axis=0, how='any')

end = time.time()
calculatedGeneralRAPIDS = end - start

print('Cálculo de transformación RAPIDS:', calculatedGeneralRAPIDS)

In [17]:
#dividir columna time
time = newData['time']
time.str.normalize_spaces()
newtime = time.str.split(pat = "-")

day = newtime[2]
day.str.normalize_spaces()
daytime = day.str.split(pat = 'T')
daytime.drop_column(1)
daytime.columns = ['day']

timenew = newtime.join(daytime, how='left')
timenew.drop_column(2)
timenew.columns = ['year','month','day']

#Unir todo al dataframe general
newData = newData.join(timenew)
newData.drop_column('time')

#eliminar filas con datos perdidos
newData.dropna(axis=0, how='any')

#resetear index
newData = newData.reset_index()
newData.drop_column('index')

In [18]:
#Informacion de todo el dataset transformado
print('Names columns:',newData.columns)
print('Dataframe empty yes or no:',newData.empty)
print('Dimension DataFrame:',newData.shape)
print('Verificar valores nulos por cada columna:',newData.isnull().sum())

print('Information DATA general again with RAPIDS:')
newData.head()

Names columns: Index(['username', 'event_source', 'host', 'referer', 'ip', 'a', 'b', 'c',
       'year', 'month', 'day'],
      dtype='object')
Dataframe empty yes or no: False
Dimension DataFrame: (77912, 11)
Verificar valores nulos por cada columna: username        0
event_source    0
host            0
referer         0
ip              0
a               0
b               0
c               0
year            0
month           0
day             0
dtype: int64
Information DATA general again with RAPIDS:


,username,event_source,host,referer,ip,a,b,c,year,month,day
0,TeresaLeon,0,0,http://opencampus.utpl.edu.ec/courses/course-v...,181.199.120.120,{'course_user_tags': {},'user_id': 13961,'org_id': 'UTPL',2017,12,11
1,TeresaLeon,1,0,http://opencampus.utpl.edu.ec/courses/course-v...,181.199.120.120,{'user_id': 13961,'org_id': 'UTPL','course_id': 'course-v1:UTPL+EFHE-Ed7+2017_DIC',2017,12,11
2,TeresaLeon,0,0,http://opencampus.utpl.edu.ec/courses/course-v...,181.199.120.120,{'user_id': 13961,'org_id': '','course_id': '',2017,12,11
3,TeresaLeon,0,0,http://opencampus.utpl.edu.ec/courses/course-v...,181.199.120.120,{'course_user_tags': {},'user_id': 13961,'org_id': 'UTPL',2017,12,11
4,monicamorocho,0,0,http://opencampus.utpl.edu.ec/courses/course-v...,186.43.211.50,{'course_user_tags': {},'user_id': 14297,'org_id': 'UTPL',2017,12,11


In [22]:
#Guardar la data entendible en un nuevo csv
import time
start = time.time()
da_data = 'data_mejora.csv'
newData.to_csv(da_data)
end = time.time()

calculated = end - start

print("Cálculo carga cudf = {}".format(calculated))

Cálculo carga cudf = 0.17021536827087402


In [24]:
nueva_data = cudf.read_csv('data_mejora.csv')
nueva_data.drop_column('Unnamed: 0')
nueva_data.head()

,username,event_source,host,referer,ip,a,b,c,year,month,day
0,TeresaLeon,0,0,http://opencampus.utpl.edu.ec/courses/course-v...,181.199.120.120,{'course_user_tags': {},'user_id': 13961,'org_id': 'UTPL',2017,12,11
1,TeresaLeon,1,0,http://opencampus.utpl.edu.ec/courses/course-v...,181.199.120.120,{'user_id': 13961,'org_id': 'UTPL','course_id': 'course-v1:UTPL+EFHE-Ed7+2017_DIC',2017,12,11
2,TeresaLeon,0,0,http://opencampus.utpl.edu.ec/courses/course-v...,181.199.120.120,{'user_id': 13961,'org_id': '','course_id': '',2017,12,11
3,TeresaLeon,0,0,http://opencampus.utpl.edu.ec/courses/course-v...,181.199.120.120,{'course_user_tags': {},'user_id': 13961,'org_id': 'UTPL',2017,12,11
4,monicamorocho,0,0,http://opencampus.utpl.edu.ec/courses/course-v...,186.43.211.50,{'course_user_tags': {},'user_id': 14297,'org_id': 'UTPL',2017,12,11


In [25]:
#Data Splitting with cuml
from cuml.preprocessing.model_selection import train_test_split

#Assign dataset general a new variables
gdf_dataX = nueva_data
gdf_dataY = nueva_data


#Divide datasets in train and test
#datatrain 80% and datatest 20%
X_train, X_test, y_train, y_test = train_test_split(gdf_dataX, gdf_dataY, train_size=0.8)

#information
print(f'Original datasets: {gdf_dataX.shape[0], gdf_dataY.shape[0]} elements')
print(f'Data X_train: {X_train.shape[0]} elements')
print(f'Data X_test: {X_test.shape[0]} elements')
print(f'Data y_train: {y_train.shape[0]} elements')
print(f'Data y_test: {y_test.shape[0]} elements')

Original datasets: (77912, 77912) elements
Data X_train: 62329 elements
Data X_test: 15583 elements
Data y_train: 62329 elements
Data y_test: 15583 elements


In [26]:
#work processing cuml with RAPIDS

#Fase Preprocessing

#Converting a categorical implementation to a numerical one
from cuml.preprocessing.LabelEncoder import LabelEncoder

le = LabelEncoder()

#select two columns potenciales for work train for xTrain
value_usernameX = le.fit_transform(X_train.username)
value_ipX = le.fit_transform(X_train.ip)

#select two columns potenciales for work train for xTest
value_usernameXT = le.fit_transform(X_test.username)
value_ipXT = le.fit_transform(X_test.ip)

#select two columns potenciales for work train for yTrain
value_usernameY = le.fit_transform(y_train.username)
value_ipY = le.fit_transform(y_train.ip)

#select two columns potenciales for work train for yTest
value_usernameYT = le.fit_transform(y_test.username)
value_ipYT = le.fit_transform(y_test.ip)

In [27]:
#Algortih LinearRegression
import cupy as cp
from cuml.preprocessing import LabelBinarizer
from cuml.linear_model import LinearRegression

#Value train-test eje Xtrain of column username
dataTest = cudf.DataFrame()
dataTest['usernameValueXtrain'] = cp.asarray(value_usernameX, dtype=cp.float32)
dataTest['ipValueXtrain'] = cp.asarray(value_ipX, dtype=cp.float32)

#Value train-test eje Ytrain of column username
new_value = cp.asarray(value_usernameY, dtype=cp.float32)
dataY = cudf.Series(new_value)

#use algorith LinearRegresssion
lr = LinearRegression(fit_intercept = True, normalize = False,
                      algorithm = "eig")

reg = lr.fit(dataTest, dataY)

print("Coefficients:")
print(reg.coef_)

print("Intercept:")
print(reg.intercept_)

Coefficients:
0    9.999990e-01
1    9.961973e-08
dtype: float32
Intercept:
0.0001983642578125


In [28]:
#Algortih RandonForest
from cuml.ensemble import RandomForestClassifier as cuRFC

dataTrainUSER = cp.asarray(value_usernameX, dtype=cp.float32)
dataTestUSER = cp.asarray(value_usernameY, dtype=cp.int32)

cuml_modelRDF = cuRFC(max_features=1.0,
                   n_bins=2,
                   n_estimators=2)

cuml_modelRDF.fit(dataTrainUSER, dataTestUSER)
cuml_predict = cuml_modelRDF.predict(dataTrainUSER)

print("Predicted labels : ", cuml_predict)

Predicted labels :  [215 215 215 ... 422 215 422]


/home/jose/anaconda3/envs/rapids/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Switching over to use the CPU predict since the GPU predict currently cannot perform multi-class classification.
  if sys.path[0] == '':


In [ ]:
#Dividir columna course_user_tags
#newcontext.columns = ['course_user_tags','user_id','org_id']
#newcontext.head(2)

#dfnewcontext = newcontext["course_user_tags"].str.split(':', expand=True)
#dfnewcontext.sort_values(0)
#dfnewcontext.drop_duplicates(1, keep="last", inplace=True)
#dfnewcontext.drop(0, inplace=True)
#dfnewcontext.columns = ['user_id']
#indexNames = dfnewcontext[dfnewcontext['user_id'] == '{}' ].index
#nuevodf = dfnewcontext.drop(indexNames, inplace=True)
#nuevodf.reset_index()